# Main built-in LCEL functions for runnables

## Contents
* .bind()
* .assign()

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

In [3]:
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo-0125")
model = ChatOpenAI(model=MODEL_GPT)

## LCEL Chain

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [5]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he has a unique habit of celebrating goals by performing a specific jump and spin move, often referred to as the "Sii!" celebration. This celebration has become iconic among fans and is characterized by Ronaldo jumping into the air, turning, and landing with his arms outstretched while shouting "Sii!" (which means "Yes!" in Spanish). Interestingly, he developed this celebration as a way to express his joy and excitement for scoring, and it has become a signature move that fans around the world associate with him. Additionally, Ronaldo\'s dedication to fitness and his rigorous training regime have allowed him to maintain peak performance, even into his late 30s, making him one of the most enduring athletes in football history.'

## Use of .bind() to add arguments to Runnable in LCEL Chain

In [7]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

In [8]:
chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano '

## Use of .bind() to call OpenAI Function in LCEL Chain

In [9]:
functions = [
    {
      "name": "soccerfacts",
      "description": "Curious facts about a soccer player",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question for the curious facts about a soccer player"
          },
          "answer": {
            "type": "string",
            "description": "The answer to the question"
          }
        },
        "required": ["question", "answer"]
      }
    }
  ]

In [10]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt
    | model.bind(function_call={"name": "soccerfacts"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [11]:
chain.invoke(input={"soccer_player": "Mbappe"})

{'question': 'What is a curious fact about Kylian Mbappé?',
 'answer': 'Kylian Mbappé was only 19 years old when he became the second youngest player to score in a World Cup match, doing so in the 2018 World Cup.'}

## The assign() function allows adding keys to chain

In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

chain = RunnableParallel({"original_input": RunnablePassthrough()})

In [13]:
chain.invoke("whatever")

{'original_input': 'whatever'}

In [14]:
def make_uppercase(arg):
    return arg["original_input"].upper()

In [15]:
chain = RunnableParallel({"original_input": RunnablePassthrough()}).assign(uppercase=RunnableLambda(make_uppercase))

In [16]:
chain.invoke("whatever")

{'original_input': 'whatever', 'uppercase': 'WHATEVER'}